In [15]:
import sys
import os
import numpy as np
import numpy.linalg as la
import torch

from ControlRF import GPController, ADPKernel, ADPRandomFeatures, ADKernel, ADRandomFeatures, VanillaKernel, VanillaRandomFeatures
from ControlRF.plots import *
from ControlRF.util import data_gen

import core
from core.controllers import QPController, LQRController, FBLinController
from core.dynamics import AffineQuadCLF
from core.systems import InvertedPendulum, DoubleInvertedPendulum

In [16]:
def main_invp(plot_sim=True, plot_errorbars=False): #d=1
    system = InvertedPendulum(0.7,0.7)
    system_est = InvertedPendulum(0.50,0.5)
    lyap_est = AffineQuadCLF.build_care(system_est, Q=np.identity(2), R=np.identity(1))
    
    alpha = 1 / max(la.eigvals(lyap_est.P))
    
    # Nominal Controller Static Slacked
    controller = QPController(system_est, system.m)
    controller.add_static_cost(np.identity(1))
    controller.add_stability_constraint(lyap_est, comp=lambda r: alpha * r, slacked=True, coeff = 1e3)
    # plot_simulation(system, controller, 'qp_controller', x_0, T=100, num_steps=1000)
    x_0 = np.array([2, 0.1])
    xs, ys, zs = data_gen(system, controller, lyap_est, x_0, 100, 1000)
    ad_rf = ADRandomFeatures(xs, ys, zs, rf_d=50)
    adp_rf = ADPRandomFeatures(xs, ys, zs, rf_d=50)
    ad_kernel = ADKernel(xs, ys, zs)
    adp_kernel = ADPKernel(xs, ys, zs)
    gps = [ad_kernel, ad_rf, adp_kernel, adp_rf]
    controllers = []
    x_0 = np.array([1, 0.1])

    for gp in gps:
        gp_controller = GPController(system_est, gp)
        gp_controller.add_static_cost(np.identity(1))
        gp_controller.add_stability_constraint(lyap_est, comp=alpha, slacked=True, beta = 1, coeff=1e5)
        controllers.append(gp_controller)
        print(f'training time for {gp.__name__}_gp is: {gp.training_time}')
        if plot_sim:
            plot_simulation(system, gp_controller, f'{gp.__name__}_controller', x_0, T=100, num_steps=1000)
            plot_closed_loop_errorbar(system, lyap_est, gp_controller, gp, x_0, cut_off=180) #cut_off=180 is good as well

    if plot_errorbars:
        plot_pred_errorbar(xs[300:350], ys[300:350], zs[300:350], gps)
        plot_pred_errorbar(xs[950:], ys[950:], zs[950:], gps)
        
        plot_all_closed_loop_errorbars(system, lyap_est, controllers, gps, x_0, cut_off=180)

    
    xs, ys, zs = data_gen(system, controller, lyap_est, x_0, 100, 1000)
ad_rf = ADRandomFeatures(xs, ys, zs, rf_d=50)
adp_rf = ADPRandomFeatures(xs, ys, zs, rf_d=50)
ad_kernel = ADKernel(xs, ys, zs)
adp_kernel = ADPKernel(xs, ys, zs)
gps = [ad_kernel, ad_rf, adp_kernel, adp_rf]
controllers = []

# plot_simulation_dip(system, controller, 'qp_controller', x_0, T=100, num_steps=1000)

for gp in gps:
    gp_controller = GPController(system_est, gp)
    controller.add_regularizer(fb_lin, 25)
    # gp_controller.add_static_cost(np.identity(2))
    gp_controller.add_stability_constraint(lyap_est, comp=alpha, slacked=True, beta = 1, coeff=1e8)
    controllers.append(gp_controller)
    
        
   
        


In [17]:
# initial_x0s = np.mgrid[.1:np.pi:1, 0:np.pi:1, 0:np.pi:1, 0:np.pi:1].reshape(4, -1).T

# system = DoubleInvertedPendulum(0.25,0.25,.7,0.7)
# system_est = DoubleInvertedPendulum(0.5,0.5,0.5,0.5)

# Q , R = 10*np.identity(4), np.identity(2)
# lyap_est = AffineQuadCLF.build_care(system_est, Q, R) 
# alpha = min(la.eigvalsh(Q)) / max(la.eigvalsh(lyap_est.P))

# lqr = LQRController.build(system, Q, R)

# # fb_lin_data = system.simulate(x_0, fb_lin, ts) #1

# controller = QPController.build_care(system, Q, R)

# controller.add_stability_constraint(lyap_est, comp=lambda r: alpha * r, slacked=True, coeff = 1e5)

# for i,x_0 in enumerate(initial_x0s):
#     if i==0:
#         xs, ys, zs = data_gen(system, controller, lyap_est, torch.from_numpy(x_0), 1, 10)
#     else:
#         x, y, z = data_gen(system, controller, lyap_est, torch.from_numpy(x_0), 10, 100)
#         xs = np.concatenate((xs,x))
#         ys = np.concatenate((ys,y))
#         zs = np.concatenate((zs,z))

In [18]:
# np.savez('data,pi,1,sim:10,100', xs=xs, ys=ys, zs=zs)

In [19]:
data = np.load('data,pi,1,sim:10,100.npz')
xs = data['xs']
ys = data['ys']
zs = data['zs']

In [20]:
    ad_rf = ADRandomFeatures(xs, ys, zs, rf_d=1200)
    adp_rf = ADPRandomFeatures(xs, ys, zs, rf_d=1200)
    ad_kernel = ADKernel(xs, ys, zs)
    adp_kernel = ADPKernel(xs, ys, zs)
    gps = [ad_kernel, ad_rf, adp_kernel, adp_rf]
    controllers = []
    x_0 = np.array([1,0,0,0])

    for gp in gps:
        gp_controller = GPController(system_est, gp)
        gp_controller.add_static_cost(np.identity(2))
        gp_controller.add_stability_constraint(lyap_est, comp=alpha, slacked=True, beta = 1, coeff=1e5)
        controllers.append(gp_controller)
        print(f'training time for {gp.__name__}_gp is: {gp.training_time}')

NameError: name 'system_est' is not defined

In [ ]:

plot_simulation_dip(system, controller, 'qp_controller', x_0, T=100, num_steps=1000)
for gp,controller in zip(gps,controllers):
    plot_simulation_dip(system, controller, f'{gp.__name__}_controller', x_0, T=100, num_steps=1000)
    plot_closed_loop_errorbar(system, lyap_est, gp_controller, gp, x_0, cut_off=180) #cut_off=180 is good as well


plot_pred_errorbar(xs[300:350], ys[300:350], zs[300:350], gps)
plot_pred_errorbar(xs[950:], ys[950:], zs[950:], gps)

plot_all_closed_loop_errorbars(system, lyap_est, controllers, gps, x_0, cut_off=180)


In [ ]:
#main_invp()
#main_double_invp()

# Q , R = 10*np.identity(4), np.identity(2)
# lyap_est = AffineQuadCLF.build_care(system_est, Q, R) 
# alpha = min(la.eigvalsh(Q)) / max(la.eigvalsh(lyap_est.P))

# lqr = LQRController.build(system, Q, R)
# fb_lin = FBLinController(system, lqr)
# # fb_lin_data = system.simulate(x_0, fb_lin, ts) #1

# controller = QPController.build_care(system, Q, R)
# controller.add_regularizer(fb_lin, 25)
# controller.add_stability_constraint(lyap_est, comp=lambda r: alpha * r, slacked=True, coeff = 1e5)

# for i,x_0 in enumerate(initial_x0s):
#     if i==0:
#         xs, ys, zs = data_gen(system, controller, lyap_est, torch.from_numpy(x_0), 10, 100)
#     else:
#         x, y, z = data_gen(system, controller, lyap_est, torch.from_numpy(x_0), 10, 100)
#         xs = np.concatenate((xs,x))
#         ys = np.concatenate((ys,y))
#         zs = np.concatenate((zs,z))